In [2]:
import numpy as np
import copy
import time
from vm_generator import *

In [10]:
overload_data, unoverload_data = InstanceGenerator(5,100)

#objective: select minimum vm mem lists

capacity = 100 #physical node CPU threshold
weight = [i[0] for i in overload_data[:]] #vm cpu usage -> weight
profit = [i[1] for i in overload_data[:]]  #vm mem usage -> value

unoverloaded_bin = [i[0] for i in unoverload_data[:]]


In [5]:
#greedy to select vm list

def greedy_select_vm(profit, weight):
    selected_vm_cpu = []
    for p, w in zip(profit, weight):
        mem_sorted, cSortByMem = (list(i) for i in zip(*sorted(zip(p,w)))) #sorted mem size and weight sorted by mem size
        
        current_weight = sum(cSortByMem)
        for w in cSortByMem:
            if current_weight <= capacity:
                break

            selected_vm_cpu.append(w)
            current_weight -= w

    print(f"Selected VM numbers: {len(selected_vm_cpu)}")
    print(f"Selected VM CPU: {selected_vm_cpu}")
    return selected_vm_cpu


In [6]:
def knapSack(W: int, wt, val, n):
    reserved = []
    m = [[0 for x in range(W + 1)] for x in range(n + 1)]
    
    for i in range(n + 1):
        for w in range (W + 1):
            if i == 0 or w == 0:
                m[i][w] = 0
            elif wt[i-1] > w:
                m[i][w] = m[i-1][w]
            else:
                m[i][w] = max(m[i-1][w],m[i-1][w-wt[i-1]]+val[i-1])
            
    result = m[n][W]
    # print(result)
    
    
    w = W
    for i in range(n,0,-1):
        if result <= 0:
            break
        if result == m[i-1][w]:
            continue
        else:
            # print(i-1,wt[i-1])
            reserved.append(wt[i-1])
            result = result - val[i-1]
            w = w-wt[i-1]
    return reserved

In [7]:
def pack(values, maxValue):
    migrateCounter = newMachineCounter = 0

    values = sorted(values, reverse=True)
    bins = copy.deepcopy(unoverloaded_bin)

    for item in values:
        # Try to fit item into a bin
        min_space = float('inf')
        min_index = -1
        for index, bin in enumerate(bins):
            #find minimum space and enough capacity to accomodate
            if sum(bin) + item <= maxValue and (maxValue-sum(bin)) < min_space:
                #print 'Adding', item, 'to', bin
                min_space = maxValue-sum(bin)
                min_index = index
                # migration_time += 1
        if min_index == -1:
            # item didn't fit into any bin, start a new bin
            #print 'Making new bin for', item
            bin = []
            bin.append(item)
            migrateCounter += 1
            bins.append(bin)
            newMachineCounter += 1
        else:
            bins[min_index].append(item)
            migrateCounter += 1



    return bins, migrateCounter, newMachineCounter

In [11]:

if __name__ == '__main__':

    def packAndShow(aList, maxValue):
        # print ('List with sum', sum(aList), 'requires at least', (sum(aList)+maxValue-1)/maxValue, 'bins')

        bins, migrateCounter, newMachineCounter = pack(aList, maxValue)
        
        print(f"Migration Times: {migrateCounter}")
        print(f"New Machine Times: {newMachineCounter}")

        print ('Solution using', len(bins), 'bins:')
        for bin in bins:
            print (bin)
    
    start = time.time()
    selected_vm_cpu = greedy_select_vm(profit, weight)
    packAndShow(selected_vm_cpu, 100)
    end = time.time()

    print(f"time: {end-start}")


Selected VM numbers: 42
Selected VM CPU: [27, 12, 21, 19, 24, 16, 29, 25, 21, 5, 29, 24, 19, 28, 24, 24, 16, 27, 30, 16, 28, 30, 29, 9, 27, 17, 19, 21, 5, 25, 19, 19, 29, 8, 21, 30, 29, 19, 10, 29, 20, 17]
Migration Times: 42
New Machine Times: 0
Solution using 60 bins:
[11, 14, 14, 18, 10]
[16, 12, 19, 10, 16, 27]
[14, 12, 10, 17, 19, 28]
[13, 18, 12, 11, 14]
[15, 11, 13, 12, 18]
[10, 13, 14, 16, 14]
[22, 10, 11, 14, 14, 29]
[17, 10, 10, 12, 13]
[15, 12, 14, 13, 15]
[12, 18, 18, 11, 14, 27]
[11, 17, 14, 17, 14, 27]
[12, 15, 13, 15, 12]
[11, 14, 24, 10, 15, 24]
[11, 13, 10, 13, 14]
[12, 11, 16, 23, 12, 21, 5]
[14, 11, 15, 23, 10, 17, 10]
[10, 17, 15, 16, 13, 29]
[19, 11, 10, 17, 16, 16, 8]
[14, 10, 12, 15, 16]
[20, 11, 10, 18, 11, 30]
[10, 16, 13, 13, 10]
[20, 10, 10, 15, 17, 28]
[13, 10, 10, 24, 15]
[16, 16, 10, 22, 10, 21, 5]
[14, 10, 10, 21, 10]
[22, 14, 11, 16, 10, 16]
[11, 24, 10, 15, 10, 30]
[14, 14, 10, 17, 20, 25]
[15, 11, 11, 10, 19]
[16, 18, 15, 12, 10, 29]
[13, 12, 26, 14, 1

In [12]:

if __name__ == '__main__':

    def packAndShow(aList, maxValue):
        # print ('List with sum', sum(aList), 'requires at least', (sum(aList)+maxValue-1)/maxValue, 'bins')

        bins, migrateCounter, newMachineCounter = pack(aList, maxValue)
        
        print(f"Migration Times: {migrateCounter}")
        print(f"New Machine Times: {newMachineCounter}")

        print ('Solution using', len(bins), 'bins:')
        for bin in bins:
            print (bin)

    start = time.time()
    selected_vm_cpu = []
    for w, p in zip(weight, profit):
        reserved = knapSack(100, w, p, len(p))
        migrate = list(w)
        
        for vm in reserved:
            migrate.remove(vm)
        selected_vm_cpu += migrate
        
    print(f"Selected VM numbers: {len(selected_vm_cpu)}")
    print(f"Selected VM CPU: {selected_vm_cpu}")
    packAndShow(selected_vm_cpu, 100)

    end = time.time()

    print(f"time: {end-start}")



Selected VM numbers: 38
Selected VM CPU: [27, 21, 19, 24, 16, 29, 25, 21, 29, 24, 19, 28, 24, 24, 16, 27, 30, 16, 28, 30, 29, 27, 17, 19, 21, 25, 19, 19, 29, 8, 21, 30, 29, 19, 10, 29, 20, 24]
Migration Times: 38
New Machine Times: 0
Solution using 60 bins:
[11, 14, 14, 18, 10]
[16, 12, 19, 10, 16, 27]
[14, 12, 10, 17, 19, 28]
[13, 18, 12, 11, 14]
[15, 11, 13, 12, 18]
[10, 13, 14, 16, 14]
[22, 10, 11, 14, 14, 29]
[17, 10, 10, 12, 13]
[15, 12, 14, 13, 15]
[12, 18, 18, 11, 14, 27]
[11, 17, 14, 17, 14, 27]
[12, 15, 13, 15, 12]
[11, 14, 24, 10, 15, 24]
[11, 13, 10, 13, 14]
[12, 11, 16, 23, 12, 24]
[14, 11, 15, 23, 10, 17, 10]
[10, 17, 15, 16, 13, 29]
[19, 11, 10, 17, 16, 16, 8]
[14, 10, 12, 15, 16]
[20, 11, 10, 18, 11, 30]
[10, 16, 13, 13, 10]
[20, 10, 10, 15, 17, 28]
[13, 10, 10, 24, 15]
[16, 16, 10, 22, 10, 21]
[14, 10, 10, 21, 10]
[22, 14, 11, 16, 10, 16]
[11, 24, 10, 15, 10, 30]
[14, 14, 10, 17, 20, 25]
[15, 11, 11, 10, 19]
[16, 18, 15, 12, 10, 29]
[13, 12, 26, 14, 10, 25]
[18, 14, 15,